<a href="https://colab.research.google.com/github/Ruheena-S/cs6910_assignment1/blob/main/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import wandb
wandb.init(project='Assignment 1', entity='cs21m053-cs21m071')


In [ ]:
from keras.datasets import fashion_mnist
import numpy as np

In [ ]:
((x_train,y_train),(x_test,y_test)) = fashion_mnist.load_data()

In [ ]:
#displaying one image from each set

imgs=[]
for i in range(0,len(set(y_train))):
  class_label=list(y_train).index(i)     #first appearance of each class label
  imgs.append(wandb.Image(x_train[class_label]))
          
wandb.log({"examples": imgs})

In [ ]:
"""
A class to create an object of one layer along with parameters to specifiy:
  input dimensions,
  number of nodes,
  activation function,
"""
class Layer:
    
    def __init__(self, inputDimension, noOfNodes, activation='',optimizer='standard',wtype = 'random'):

        # Optimization Alogorithm for gradient Descent
        if optimizer == 'standard':
            self.optimizer = self.standard
        elif optimizer == 'momentumGD':
            self.optimizer = self.momentumGD
        elif optimizer == 'rmsprop':
            self.optimizer = self.rmsprop 
        elif optimizer == 'adam':
            self.optimizer = self.adam
        elif optimizer == 'sgd':
            self.optimizer = self.sgd
        elif optimizer == 'nadam':
            self.optimizer = self.nadam
        elif optimizer == 'nesterov':
            self.optimizer = self.nesterov


        # Activation Function for each layer
        if activation == 'sigmoid':
            self.activation = activation
            self.activationForward = self.sigmoid
            self.activationBackward = self.sigmoidGrad
        elif activation == 'relu':
            self.activation = activation
            self.activationForward = self.relu
            self.activationBackward = self.relugrad

        elif activation == 'tanh':
            self.activation = activation
            self.activationForward = self.tanh
            self.activationBackward = self.tanhgrad

        else:
            self.activation = 'softmax'
            self.activationForward = self.softmax
            self.activationBackward = self.softmaxGrad

        self.weights, self.bias = self.initialize(inputDimension, noOfNodes,activation,wtype = wtype)
        self.prevMW, self.prevMb = np.zeros([noOfNodes,inputDimension]),np.zeros([noOfNodes, 1])
        self.prevVW, self.prevVb = np.zeros([noOfNodes,inputDimension]),np.zeros([noOfNodes, 1])
        self.opt = optimizer

        
    
    # Initialize the layer with some random weights and bias
    def initialize(self, noOfInputFeatures, noOfNodes,activation,wtype):
        np.random.seed(1)
        #limit = np.sqrt(2/(noOfInputFeatures+noOfNodes))
        if wtype == 'random':
          weights = np.random.normal(0.0,0.5,size=(noOfNodes, noOfInputFeatures))
          bias = np.ones([noOfNodes, 1])
        
        else:
           weights = np.random.uniform(-(1/np.sqrt(noOfNodes)), (1/np.sqrt(noOfNodes)), size=(noOfNodes, noOfInputFeatures))
           bias = np.ones([noOfNodes, 1])
        return weights, bias

    # Sigmoid activation function
    def sigmoid(self, Z):
        A = 1 / (1 + np.exp(-Z))
        return A

    # Differential of sigmoid function with chain rule applied
    def sigmoidGrad(self, dA):
        s = 1 / (1 + np.exp(-self.prevZ))
        dZ = dA * s * (1 - s)
        return dZ

    #Tanh Activation Function
    def tanh(self,Z):
        return np.tanh(Z)

    def tanhgrad(self,dA):
        s = self.tanh(self.prevZ)
        return dA*(1-(s**2))

    #Relu Activation function
    def relu(self,Z):
        A = np.maximum(0,Z)
        return A

    def relugrad(self,dA):
        s = np.maximum(0,self.prevZ)
        t = 1. * dA * (s>0) 
        return t

   

    #SoftMax Activation Function
    def softmax(self,Z):
      expZ = np.exp(Z - np.max(Z))
      A = expZ / expZ.sum(axis=0, keepdims=True)
      return A

    def softmaxGrad(self,dA):
      return dA

    


    # Take's the input vector A and does the forward pass using weights and bias
    def forward(self, A):

      if self.opt != 'nesterov':
        Z = np.dot(self.weights,A) +self.bias
      else:
        Z = np.dot(self.weights-0.9*self.prevVW, A) + (self.bias-0.9*self.prevVb)
      self.prevZ = Z
      self.prevA = A
      A = self.activationForward(Z)
      return A

      
    
    # Take's the input vector derivative of A (dA) and does the backward pass
    def backward(self, dA):
        dZ = self.activationBackward(dA)
        m = self.prevA.shape[1]
        self.dW = 1 / m * np.dot(dZ, self.prevA.T) 
        self.db = 1 / m * np.sum(dZ, axis=1, keepdims=True)
        prevdA = np.dot(self.weights.T, dA)
        return prevdA

    
    # Update's the weights and bias with the passed learning_rate
    def standard(self, learning_rate,t):
        self.weights = self.weights - learning_rate * self.dW
        self.bias = self.bias - learning_rate * self.db
    
    def momentumGD(self, learning_rate,t, moving_rate=0.9):
        self.prevMW = moving_rate * self.prevMW + learning_rate * self.dW
        self.prevMb = moving_rate * self.prevMb + learning_rate * self.db
        self.weights = self.weights - self.prevMW
        self.bias = self.bias - self.prevMb
        

    def rmsprop(self, learning_rate,t, moving_rate = 0.9):
        self.prevVW = moving_rate * self.prevVW + (1-moving_rate) * (self.dW**2)
        self.prevVb = moving_rate * self.prevVb + (1-moving_rate) * (self.db**2)
        for i in self.prevVW:
          i[i<0] = 1e-9
        self.prevVb[self.prevVb<0] = 1e-9 
        self.weights = self.weights - (learning_rate/np.sqrt(self.prevVW+(1e-8))) * self.dW
        self.bias = self.bias - (learning_rate/np.sqrt(self.prevVb+(1e-8))) * self.db

    def sgd(self, dA,learning_rate = 0.001,t = 0):
        dZ = self.activationBackward(dA)
        #print(dZ.shape)
        prevdA = np.dot(self.weights.T, dZ)
        m = self.prevA.shape[1]
        for i in range(m):
          self.dW = 1 / m * np.dot(dZ[:,i:i+1], self.prevA[:,i:i+1].T)   #arr[:,2]
          self.db = 1 / m * (dZ[:,i:i+1])
          self.weights = self.weights - (learning_rate * self.dW)
          self.bias = self.bias - (learning_rate * self.db)
        #prevdA = np.dot(self.weights.T, dZ)
        return prevdA

    def nesterov(self,learning_rate,moving_rate = 0.9,t = 0):
        
        self.prevVW = moving_rate * self.prevVW + learning_rate * self.dW
        self.weights = self.weights - self.prevVW

        self.bl = self.bias - moving_rate * self.prevVb
        self.prevVb = moving_rate * self.prevVb

    def adam(self,learning_rate ,t, beta1 = 0.9, beta2 = 0.999):
        self.prevMW = beta1 * self.prevMW + (1-beta1)*self.dW
        self.prevMb = beta1 * self.prevMb + (1-beta1)*self.db

        self.prevVW = beta2 * self.prevVW + (1-beta2)*np.square(self.dW)
        self.prevVb = beta2 * self.prevVb + (1-beta2)*np.square(self.db)

        self.prevMWHat = self.prevMW/(1-beta1)
        self.prevMbHat = self.prevMb/(1-beta1)

        self.prevVWHat = self.prevVW/(1-beta2)
        self.prevVbHat = self.prevVb/(1-beta2)

        self.weights = self.weights - learning_rate * np.divide(self.prevMWHat,np.sqrt(self.prevVWHat+(1e-8)))
        self.bias = self.bias - learning_rate * np.divide(self.prevMbHat,np.sqrt(self.prevVbHat+(1e-8)))
        
    def nadam(self,learning_rate ,t, beta1 = 0.9, beta2 = 0.999):
        self.prevMW = beta1 * self.prevMW + (1-beta1)*self.dW
        self.prevMb = beta1 * self.prevMb + (1-beta1)*self.db

        self.prevVW = beta2 * self.prevVW + (1-beta2)*np.square(self.dW)
        self.prevVb = beta2 * self.prevVb + (1-beta2)*np.square(self.db)

        self.prevMWHat = (beta1 * self.prevMW / (1-beta1)) + self.dW
        self.prevMbHat = (beta1 * self.prevMb / (1-beta1)) + self.db

        self.prevVWHat = (beta2 * self.prevVW) / (1-beta2)
        self.prevVbHat = (beta2 * self.prevVb) / (1-beta2)

        self.weights = self.weights - learning_rate * np.divide(self.prevMWHat,np.sqrt(self.prevVWHat+(1e-8)))
        self.bias = self.bias - learning_rate * np.divide(self.prevMbHat,np.sqrt(self.prevVbHat+(1e-8)))
        



In [ ]:
"""
Neural Network which keeps consists of all layers and some helper functions
"""

class NeuralNetwork:
    
    def __init__(self, layers_size,epochs,learning_rate, optimizer = 'standard', activation = 'sigmoid',wtype = 'random', loss='cross_entropy'):
        self.layers=[]
        self.layers_size = layers_size
        self.optimizer = optimizer
        self.activation = activation
        self.wtype = wtype
        self.epochs = epochs
        self.learning_rate = learning_rate
        if loss=='cross_entropy':
            self.lossFunction = self.cross_entropy
            self.lossBackward = self.cross_entropy_Grad
        elif loss == 'mean_square':
            self.lossFunction = self.meanSquareError
            self.lossBackward = self.meanSquareErrorGrad
        else:
            print('Invalid loss function')
        self.loss=loss

    # add layer to the NN with input dimensions, nodes & activation function
    def addLayer(self, inputDimension=None, noOfNodes=1, activation='',wtype = 'random'):
        if (inputDimension is None):
            if (len(self.layers) == 0):
                print('Invalid number of layers')
            inputDimension = self.layers[-1].outputDimension()
        layer = Layer(inputDimension, noOfNodes, activation,optimizer = self.optimizer, wtype=wtype)
        self.layers.append(layer)

    def meanSquareError(self, Y, A):
        loss = np.square(Y - A)
        m = Y.shape[1]
        cost = 1 / m * np.sum(loss)
        return np.squeeze(cost)
    
    # Compute's mean square grad error, Y is true value & A is predicted value
    def meanSquareErrorGrad(self, Y, A):
        dA = -2 * (Y - A)
        return dA

    # Compute's cross entropy error where Y is true value and A is predicted value
    def cross_entropy(self, Y, A):
        m = Y.shape[1]
        cost = -(1/m)*np.sum(Y*np.log(A))
        return np.squeeze(cost)
    
    # Compute's cross entropy grad error, Y is true value & A is predicted value
    def cross_entropy_Grad(self, Y, A):
        dA = A-Y
        return dA

    # Wrapper function to get the cost or loss value of the predicted values
    def cost(self, Y, A):
        return self.lossFunction(Y, A)

    # Forward pass the input vector X through all layers
    def forward(self, X):
        x = np.copy(X)
        for layer in self.layers:
            x = layer.forward(x)
        return x
            
    # Backward pass the true values and predicted values in reverse direction
    def backward(self, Y, A):
        dA = self.lossBackward(Y, A)
        #print(dA.shape)
        if self.optimizer != 'sgd':
            for layer in reversed(self.layers):
                dA = layer.backward(dA)
        else:
            for layer in reversed(self.layers):
                dA = layer.sgd(dA,learning_rate = self.learning_rate)
    
    # Update weights and compute's gradient descent of all layers
    def update(self, learning_rate=0.01,t=0):
        for layer in self.layers:
            layer.optimizer(learning_rate,t=0)

    def fit(self,x,y,batch_size = 32):

        if self.activation=='relu':
          self.wtype = 'xavier'
        
        for i in range(1,len(self.layers_size)-1):
          self.addLayer(inputDimension=self.layers_size[i-1], noOfNodes=self.layers_size[i], activation=self.activation, wtype = self.wtype)

        
        # Output layer with softmax activation
        self.addLayer(inputDimension=self.layers_size[-2], noOfNodes=self.layers_size[-1], activation='softmax', wtype = self.wtype)

        y_hot = np.zeros([len(set(y)),len(y)])

        for i in range(y_hot.shape[1]):
          y_hot[y[i]][i] = 1

        for i in range(self.epochs):
          
          acc =0
          cross_entropy= 0

          
          #avoiding gradient vanishing for Relu
          if self.activation =='relu':
            self.learning_rate/=15
          
          for j in range(0,x.shape[0],batch_size):
            
            xb = x[j:j+batch_size]
            yb = y[j:j+batch_size]
            y_hotb = y_hot[:,j:j+batch_size]
            xb = xb.reshape(xb.shape[0],xb.shape[1]*xb.shape[2]).T
            xb = xb-np.min(xb)/np.max(xb)-np.min(xb)


            #Neural Network Running
            if self.optimizer == 'sgd':
              A = self.forward(xb)
              #print("fit")
              self.backward(y_hotb,A)
            else:
              A = self.forward(xb)
              self.backward(y_hotb,A)
              self.update(learning_rate=self.learning_rate,t= i+1)


            #Calculating Cross Entropy and Accuracy
            y_pred = np.argmax(A,axis = 0)
            acc += (yb == y_pred).sum()
            
            cross_entropy += -(y_hotb * np.log(A)).sum()
           
          
          print("After ",i+1,"iterations:")
          print("cross entropy:",cross_entropy/len(y))
          print("accuracy:",acc/len(y))

    def predict(self,x):
        x = x.reshape(x.shape[0],x.shape[1]*x.shape[2]).T
        A = self.forward(x)
        A = np.argmax(A,axis=0)
        return A


In [ ]:
np.random.seed(1)
model = NeuralNetwork(layers_size = [784,128,128,128,10],epochs = 5,learning_rate = 0.001,loss='cross_entropy',activation = 'sigmoid', optimizer = 'sgd', wtype='xavier' )


from sklearn.model_selection import train_test_split
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,train_size = 0.9, test_size = 0.1, random_state=10)
y_pred = model.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: RuntimeWarning: overflow encountered in exp
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:75: RuntimeWarning: overflow encountered in exp


After  1 iterations:
cross entropy: 2.317642903305737
accuracy: 0.1245
After  2 iterations:
cross entropy: 2.2909727619613984
accuracy: 0.17224074074074075
After  3 iterations:
cross entropy: 2.2823912148738
accuracy: 0.2245185185185185
After  4 iterations:
cross entropy: 2.272656365906374
accuracy: 0.2773888888888889
After  5 iterations:
cross entropy: 2.260763070673871
accuracy: 0.33005555555555555


In [ ]:
y_p = model.predict(x_val)
print((y_p == y_val).mean()*100)


37.46666666666666


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:70: RuntimeWarning: overflow encountered in exp
